Vandermonde matrix

\begin{align}V=\begin{bmatrix}
1 & \alpha_{1} & \alpha_{1}^{2}& \dots &\alpha_{1}^{n-1}\\&\alpha_{2}&\alpha_{2}^{2}&\dots &\alpha_{2}^{n-1}\\&\alpha_{3}&\alpha_{3}^{2}&\dots &\alpha_{3}^{n-1}\\ \vdots &\vdots &\vdots &\ddots &\vdots \\&\alpha_{m}&\alpha_{m}^{2}&\dots &\alpha_{m}^{n-1}
\end{bmatrix}\end{align}


In [6]:
using PyCall

In [7]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/Users/ryunagai/.julia/conda/3/lib/python3.7/site-packages/numpy/__init__.py'>

In [13]:
np.vander(1:5, increasing=true)

5×5 Array{Int64,2}:
 1  1   1    1    1
 1  2   4    8   16
 1  3   9   27   81
 1  4  16   64  256
 1  5  25  125  625

In [16]:
function vander(x::AbstractVector{T},  n=length(x)) where T
    m = length(x)
    V = Matrix{T}(undef, m, n)
    for j = 1:m
        V[j,1] = one(x[j])
    end
    for i= 2:n
        for j = 1:m
            V[j,i] = x[j] * V[j,i-1]
            end
        end
    return V
end

vander(1:5)

5×5 Array{Int64,2}:
 1  1   1    1    1
 1  2   4    8   16
 1  3   9   27   81
 1  4  16   64  256
 1  5  25  125  625

In [22]:
function vander(x::AbstractVector{T},  n=length(x)) where T
    m = length(x)
    V = Matrix{T}(undef, m, n)
    for j = 1:m
        V[j,1] = one(x[j])
    end
    for i= 2:n
        for j = 1:m
            V[j,i] = x[j] * V[j,i-1]
            end
        end
    return V
end

vander(1:5)

5×5 Array{Int64,2}:
 1  1   1    1    1
 1  2   4    8   16
 1  3   9   27   81
 1  4  16   64  256
 1  5  25  125  625

In [29]:
func(x,y) = x^2 + y
@time func(1,2)
@time func(1,2)

  0.004760 seconds (1.18 k allocations: 64.298 KiB)
  0.000003 seconds (4 allocations: 160 bytes)


3

In [30]:
@time func(1,2)
@time func(1.3,4.8) # compiled for first float input
@time func(1.3,4.8)

  0.000003 seconds (4 allocations: 160 bytes)
  0.005407 seconds (1.17 k allocations: 64.035 KiB)
  0.000002 seconds (5 allocations: 176 bytes)


6.49

In [31]:
struct Anything
    value::Any
end

operation(x::Number) = x^2 + sqrt(x)
operation(x::Anything) = x.value^2 + sqrt(x.value)

operation (generic function with 2 methods)

In [32]:
using BenchmarkTools

┌ Info: Recompiling stale cache file /Users/ryunagai/.julia/compiled/v1.0/BenchmarkTools/ZXPQo.ji for BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1190


In [33]:
@btime operation(2);
@btime operation(2.0);

x = Anything(2.0)
@btime operation($x);

  1.419 ns (0 allocations: 0 bytes)
  1.420 ns (0 allocations: 0 bytes)
  56.286 ns (3 allocations: 48 bytes)


In [42]:
f1(x::Int) = x + 1
f2(x::Int) = x + 2

function f_slow(x::Int, p::Bool)
    if p                                # check depends on the value of p
        return f1(x)
    else
        return f2(x)
    end
end

abstract type Boolean end
struct True <: Boolean end # type domain true
struct False <: Boolean end # type domain false

function f_fast(x::Int, p::Boolean)
    if typeof(p) == True                # check solely based on the type of p
        return f1(x)
    else
        return f2(x)
    end
end

f_fast (generic function with 1 method)

In [44]:
@time f_slow(1, false)
@time f_fast(1, True())

  0.000003 seconds (4 allocations: 160 bytes)
  0.000025 seconds (4 allocations: 160 bytes)


2